In [1]:
import multiprocessing as mp
import sys
# import mss
sys.path.append('../')
from mss import visreader as mvis
from mss import mssmain as msm
from mss import align
from mss import frag
from mss import dm
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
import peakutils
import scipy
import itertools

/home/hack_summer/miniconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning:

Trying to unpickle estimator DecisionTreeClassifier from version 0.22.2.post1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.

/home/hack_summer/miniconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning:

Trying to unpickle estimator RandomForestClassifier from version 0.22.2.post1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.



In [15]:
bin_count = int(np.log(16/1)/np.log(2)+1)
np.logspace(0,bin_count-1,bin_count,base=2)

array([ 1.,  2.,  4.,  8., 16.])

In [7]:
bin_count

4.0

In [3]:
np.logspace(0,3,4,base=2)

array([1., 2., 4., 8.])

In [8]:
dummydata = [[1,5,0],[0,2,4],[9,9,9]]
df = pd.DataFrame(dummydata, columns=['test1','test2','test3'])
insert_col = ['ex_1','ex_2','ex_3']
df[insert_col] = pd.DataFrame([np.zeros(len(insert_col))], index=df.index)
df

,test1,test2,test3,ex_1,ex_2,ex_3
0,1,5,0,0.0,0.0,0.0
1,0,2,4,0.0,0.0,0.0
2,9,9,9,0.0,0.0,0.0


In [6]:
import multiprocessing


def worker(procnum, return_dict):
    """worker function"""
    print(str(procnum) + " represent!")
    return_dict[procnum] = procnum


if __name__ == "__main__":
    manager = multiprocessing.Manager()
    return_dict = manager.dict()
    jobs = []
    for i in range(5):
        p = multiprocessing.Process(target=worker, args=(i, return_dict))
        jobs.append(p)
        p.start()

    print(return_dict.values())    
        
    for proc in jobs:
        proc.join()
        
    print(return_dict.values())    

0 represent!
1 represent!
2 represent!
3 represent!
4 represent!
[0, 1, 2, 3]
[0, 1, 2, 3, 4]


In [8]:
return_dict.values()[0]

0

In [1]:
global test
test = []

tlist = [1,2,3,4,5,6,7,8,9]

def tfunc(n):
    return test+n

for t in tlist:
    test = t
    print(list(map(tfunc, [1,2,3])))
    
    

[2, 3, 4]
[3, 4, 5]
[4, 5, 6]
[5, 6, 7]
[6, 7, 8]
[7, 8, 9]
[8, 9, 10]
[9, 10, 11]
[10, 11, 12]


In [32]:
path = '../example_data/ex_1.mzML'
scans = msm.get_scans(path)

In [17]:
msm.peak_pick(scans,187.0512,400)
err=50

In [33]:
def peak_pick(input_mz, mzml_scans=scans, error=err, enable_score=True, peak_thres=0.01,
              peakutils_thres=0.02, min_d=1, rt_window=1.5,
              peak_area_thres=1e5, min_scan=5, max_scan=200, max_peak=5,
              overlap_tol=15, sn_detect=15, rt=None):
    '''
    The function is used to detect peak for given m/z's chromatogram
    error: in ppm
    enable_score: option to enable the RF model
    peak_thres: base peak tolerance
    peakutils_thres: threshold from peakutils, may be repeated with peak_thres
    min_d: peaktuils parameter
    rt_window: window for integration only, didn't affect detection
    peak_area_thres: peak area limitation
    min_scan: min scan required to be detected as peak
    max_scan: max scan limit to exclude noise
    max_peak: max peak limit for selected precursor
    overlap_tot: overlap scans for two peaks within the same precursor
    sn_detect: scan numbers before/after the peak for sn calculation
    '''
    if not rt:
        rt = [i.scan_time[0] for i in mzml_scans]
    intensity = msm.ms_chromatogram_list(mzml_scans, input_mz, error)

    # Get rt_window corresponding to scan number
    scan_window = int(
        (rt_window / (rt[int(len(intensity) / 2)] -
                      rt[int(len(intensity) / 2) - 1])))
    rt_conversion_coef = np.diff(rt).mean()
    # Get peak index
    indexes = peakutils.indexes(intensity, thres=peakutils_thres,
                                min_dist=min_d)

    result_dict = {}

    # dev note: boundary detection refinement
    for index in indexes:
        h_range = index
        l_range = index
        base_intensity = peak_thres * intensity[index]
        half_intensity = 0.5 * intensity[index]

        # Get the higher and lower boundary
        while intensity[h_range] >= base_intensity:
            h_range += 1
            if h_range >= len(intensity) - 1:
                break
            if intensity[h_range] < half_intensity:  
                if h_range - index > 4:  
                    # https://stackoverflow.com/questions/55649356/
                    # how-can-i-detect-if-trend-is-increasing-or-
                    # decreasing-in-time-series as alternative
                    x = np.linspace(h_range - 2, h_range, 3)
                    y = intensity[h_range - 2: h_range + 1]
                    (_slope, _intercept, r_value,
                     _p_value, _std_err) = scipy.stats.linregress(x, y)
                    if abs(r_value) < 0.6:
                        break
        while intensity[l_range] >= base_intensity:
            l_range -= 1
            if l_range <= 1:
                break
            # Place holder for half_intensity index
            # if intensity[l_range] < half_intensity:
            #     pass

        # Output a range for the peak list
        # If len(intensity) - h_range < 4:
        #     h_range = h_range + 3
        peak_range = []
        if h_range - l_range >= min_scan:
            if rt[h_range] - rt[l_range] <= rt_window:
                peak_range = intensity[l_range:h_range]
            else:
                if index - scan_window / 2 >= 1:
                    l_range = int(index - scan_window / 2)
                if index + scan_window / 2 <= len(intensity) - 1:
                    h_range = int(index + scan_window / 2)
                peak_range = intensity[l_range:h_range]
                # print(index + scan_window)

        # Follow Agilent S/N document
        width = rt[h_range] - rt[l_range]
        if len(peak_range) != 0:
            height = max(peak_range)
            hw_ratio = round(height / width, 0)
            neighbour_blank = (intensity[
                l_range - sn_detect: l_range] +
                intensity[h_range: h_range +
                          sn_detect + 1])
            noise = np.std(neighbour_blank)
            if noise != 0:
                sn = round(height / noise, 3)
            elif noise == 0:
                sn = 0

        # Additional global parameters
        # 1/2 peak range
        h_loc = index
        l_loc = index
        while intensity[h_loc] > half_intensity:
            h_loc += 1
            if h_loc >= len(intensity) - 1:
                break
        while intensity[l_loc] > half_intensity and l_loc > 0:
            l_loc -= 1

        # Intergration based on the simps function
        if len(peak_range) >= min_scan:
            integration_result = scipy.integrate.simps(peak_range)
            if integration_result >= peak_area_thres:
                # https://doi.org/10.1016/j.chroma.2010.02.010
                background_area = (h_range - l_range) * height
                ab_ratio = round(integration_result / background_area, 3)
                if enable_score is True:
                    h_half = h_loc + \
                        (half_intensity - intensity[h_loc]) / \
                        (intensity[h_loc - 1] - intensity[h_loc])
                    l_half = l_loc + \
                        (half_intensity - intensity[l_loc]) / \
                        (intensity[l_loc + 1] - intensity[l_loc])
                    # when transfer back use rt[index] instead
                    mb = (height - half_intensity) / \
                        ((h_half - index) * rt_conversion_coef)
                    ma = (height - half_intensity) / \
                        ((index - l_half) * rt_conversion_coef)
                    w = rt[h_range] - rt[l_range]
                    t_r = (h_half - l_half) * rt_conversion_coef
                    l_width = rt[index] - rt[l_range]
                    r_width = rt[h_range] - rt[index]
                    assym = r_width / l_width
                    # define constant -- upper case
                    var = (w ** 2 / (1.764 * ((r_width / l_width)
                           ** 2) - 11.15 * (r_width / l_width) + 28))
                    x_peak = [w, t_r, l_width, r_width, assym,
                              integration_result, sn, hw_ratio, ab_ratio,
                              height, ma, mb, ma + mb, mb / ma, var]
                    x_input = np.asarray(x_peak)
                    # score = np.argmax(Pmodel.predict(x_input.reshape(1,-1)))
                    # for tensorflow
                    score = 1
                elif enable_score is False:
                    score = 1

                # appending to result
                if len(result_dict) == 0:
                    (result_dict.update(
                     {index: [l_range, h_range,
                              integration_result, sn, score]}))
                # Compare with previous item
                # * get rid of list()
                elif integration_result != list(result_dict.values())[-1][2]:
                    # test python 3.6 and 3.7
                    s_window = abs(index - list(result_dict.keys())[-1])
                    if s_window > overlap_tol:
                        (result_dict.update(
                         {index: [l_range, h_range, integration_result,
                                  sn, score]}))
    # If still > max_peak then select top max_peak results
    if len(result_dict) > max_peak:
        result_dict = dict(sorted(result_dict.items(),
                                  key=lambda x: x[1][2], reverse=True))
        result_dict = dict(itertools.islice(result_dict.items(), max_peak))

    return result_dict

print(peak_pick(299.1765))


{1395: [1259, 1530, 11141765.265787758, 168.969, 1], 1379: [1243, 1514, 11024315.757649738, 39.016, 1], 1411: [1275, 1546, 10521883.167805988, 4.484, 1], 1353: [1217, 1488, 10279178.235351562, 8.623, 1], 1336: [1200, 1471, 9999682.219075521, 26.276, 1]}


In [ ]:
def test(n=testlist):
    return n+n**2+n**3

testlist = [1,2,3,4,5,6,7,8,9]

with mp.Pool() as pool:
    res = pool.map(test,testlist)
    print(res)

In [2]:
peak_dict = [{380: [379, 491, 394209.58401489246, 29.902, 1, 100.1120357442187], 482: [379, 491, 394209.58401489246, 29.902, 1, 100.1120357442187], 416: [379, 482, 383281.9418945312, 23.087, 1, 100.1120357442187], 399: [379, 483, 383260.9562886555, 24.667, 1, 100.1120357442187], 432: [379, 478, 375978.6517740885, 18.453, 1, 100.1120357442187]}, {167: [163, 252, 144132.67783610022, 7.449, 1, 103.95618782860778], 194: [163, 231, 120315.50900014241, 5.286, 1, 103.95618782860778], 214: [163, 248, 140641.55963134763, 7.509, 1, 103.95618782860778], 230: [163, 252, 144132.67783610022, 7.449, 1, 103.95618782860778]}, {30: [29, 70, 124077.04874674478, 4.614, 1, 104.10599107350343], 116: [87, 130, 143420.04223632812, 7.537, 1, 104.10599107350343], 134: [132, 172, 121536.43308512369, 2.665, 1, 104.10599107350343], 370: [369, 404, 102039.20190429686, 4.5, 1, 104.10599107350343], 441: [440, 499, 167495.08341471353, 5.342, 1, 104.10599107350343]}, {134: [131, 339, 602878.4699707031, 3.496, 1, 104.10807319332488], 116: [1, 131, 407473.540242513, 7.651, 1, 104.10807319332488], 1: [0, 109, 320440.88435872394, 2.757, 1, 104.10807319332488], 18: [1, 109, 318227.43761189777, 2.757, 1, 104.10807319332488], 340: [339, 421, 242935.97572835285, 6.845, 1, 104.10807319332488]}, {109: [99, 151, 8400665.29119873, 570.177, 1, 104.9925547293939]}, {111: [110, 116, 1460484.0494791665, 2.907, 1, 104.99465458048847]}, {108: [100, 137, 185810.2579345703, 35.078, 1, 105.994726757684], 134: [100, 138, 186370.50796000162, 64.103, 1, 105.994726757684]}, {108: [102, 129, 161483.51025390625, 9.21, 1, 105.99684665221915]}, {158: [154, 256, 102248.36340332031, 6.626, 1, 107.07068719210373]}, {260: [256, 313, 103740.08129882814, 3.951, 1, 114.94899799749963], 396: [393, 499, 191070.6802520752, 8.57, 1, 114.94899799749963]}, {51: [47, 87, 192901.17651367188, 2.945, 1, 117.11171963998481], 332: [330, 369, 141338.69962565103, 4.584, 1, 117.11171963998481], 388: [385, 419, 143335.98323567706, 4.353, 1, 117.11171963998481], 433: [431, 490, 284436.5852864583, 3.625, 1, 117.11171963998481]}, {363: [227, 498, 1084626.318847656, 6.854, 1, 117.11406187437761], 379: [243, 499, 1046168.3783569336, 8.165, 1, 117.11406187437761], 346: [210, 481, 1040740.9916178384, 4.339, 1, 117.11406187437761], 330: [194, 465, 1005228.614501953, 4.254, 1, 117.11406187437761], 396: [260, 499, 991239.3081054686, 8.901, 1, 117.11406187437761]}, {356: [220, 491, 1671707.081705729, 11.875, 1, 119.08150577290671], 340: [204, 475, 1654565.1661783855, 9.278, 1, 119.08150577290671], 373: [237, 499, 1621420.1640625, 15.87, 1, 119.08150577290671], 323: [187, 458, 1620633.76953125, 8.308, 1, 119.08150577290671], 307: [171, 442, 1598573.595703125, 9.671, 1, 119.08150577290671]}, {205: [204, 266, 246873.40472412106, 3.098, 1, 120.9803363543696], 224: [204, 268, 252581.2128702799, 3.078, 1, 120.9803363543696]}, {359: [162, 407, 722038.4838867186, 16.029, 1, 120.98275596109667], 394: [162, 407, 722038.4838867186, 16.029, 1, 120.98275596109667], 329: [162, 405, 720258.3876342772, 15.486, 1, 120.98275596109667], 377: [162, 405, 720258.3876342772, 15.486, 1, 120.98275596109667], 313: [162, 393, 706635.8227539062, 13.949, 1, 120.98275596109667]}, {296: [160, 431, 6390423.988281248, 21.696, 1, 121.05294561069037], 277: [141, 412, 6377732.884765623, 11.566, 1, 121.05294561069037], 313: [177, 448, 6363724.227213541, 15.793, 1, 121.05294561069037], 331: [195, 466, 6357848.328125, 19.165, 1, 121.05294561069037], 347: [211, 482, 6343720.557942707, 20.361, 1, 121.05294561069037]}, {164: [163, 332, 271935.1287638346, 9.923, 1, 122.98075244306195], 291: [163, 332, 271935.1287638346, 9.923, 1, 122.98075244306195], 269: [163, 325, 266604.4825820923, 10.038, 1, 122.98075244306195], 251: [163, 285, 228430.4838053385, 6.725, 1, 122.98075244306195], 234: [163, 279, 221693.73914591467, 6.574, 1, 122.98075244306195]}, {48: [46, 121, 182360.0284016927, 15.671, 1, 124.02309205608452], 102: [46, 114, 161200.22105916342, 14.527, 1, 124.02309205608452]}, {95: [94, 129, 119165.20247395833, 15.604, 1, 124.02557251792561], 130: [129, 188, 111862.35351562501, 3.878, 1, 124.02557251792561]}, {420: [417, 479, 151470.16677347815, 11.821, 1, 136.08597380500234], 452: [417, 463, 120760.09730021158, 7.681, 1, 136.08597380500234], 469: [417, 479, 151470.16677347815, 11.821, 1, 136.08597380500234]}, {157: [21, 292, 3717710.792643229, 128.864, 1, 140.00337963129002], 173: [37, 308, 3712228.6499023438, 141.943, 1, 140.00337963129002], 207: [71, 342, 3706747.2111002607, 96.855, 1, 140.00337963129002], 190: [54, 325, 3706405.1311848955, 139.016, 1, 140.00337963129002], 224: [88, 359, 3697475.2045898433, 124.125, 1, 140.00337963129002]}, {56: [42, 136, 130771.79369099933, 4.78, 1, 141.11377551527187], 147: [146, 272, 184630.04525756836, 4.194, 1, 141.11377551527187], 279: [278, 415, 206497.58719889322, 4.406, 1, 141.11377551527187], 325: [278, 346, 102776.51195271808, 6.763, 1, 141.11377551527187], 341: [278, 415, 206497.58719889322, 4.406, 1, 141.11377551527187]}, {278: [277, 474, 279022.6078694662, 5.048, 1, 142.0339933219424]}, {337: [335, 437, 159684.1484883626, 5.352, 1, 145.00760819377555]}, {347: [211, 482, 604257.2758789062, 8.002, 1, 151.0975719441326], 280: [144, 415, 600675.4518229167, 8.187, 1, 151.0975719441326], 329: [193, 464, 600673.3092447915, 8.281, 1, 151.0975719441326], 313: [177, 448, 599727.6723632812, 6.116, 1, 151.0975719441326], 264: [128, 399, 598779.9373779297, 7.242, 1, 151.0975719441326]}, {2: [1, 102, 152666.16597493488, 0, 1, 153.08995180864605]}, {267: [131, 402, 2016999.8829752603, 4.794, 1, 155.97340038451878], 315: [179, 450, 1992740.1170247393, 15.969, 1, 155.97340038451878], 299: [163, 434, 1985795.2858072917, 14.455, 1, 155.97340038451878], 283: [147, 418, 1984986.4578450522, 7.175, 1, 155.97340038451878], 332: [196, 467, 1979785.0126953125, 14.245, 1, 155.97340038451878]}, {151: [146, 286, 1041497.397257487, 3.062, 1, 155.97651985252642], 312: [311, 403, 648793.1442871093, 6.193, 1, 155.97651985252642], 404: [403, 472, 484892.2151692708, 4.501, 1, 155.97651985252642], 61: [60, 103, 265862.75935872394, 2.527, 1, 155.97651985252642], 34: [29, 60, 198340.46468098962, 4.671, 1, 155.97651985252642]}, {188: [52, 323, 3103052.75, 84.297, 1, 158.01427109861655], 170: [34, 305, 3102411.873046875, 72.513, 1, 158.01427109861655], 207: [71, 342, 3092290.4156901035, 68.195, 1, 158.01427109861655], 153: [17, 288, 3090528.9215494785, 79.602, 1, 158.01427109861655], 223: [87, 358, 3090137.329427083, 72.026, 1, 158.01427109861655]}, {104: [100, 156, 935042.7522989908, 20.867, 1, 158.9969930423098], 150: [100, 157, 940153.7039388021, 21.018, 1, 158.9969930423098]}, {18: [16, 63, 131523.142985026, 6.288, 1, 159.00017298217065], 64: [63, 105, 159193.52563476562, 2.397, 1, 159.00017298217065], 117: [116, 125, 183086.53710937497, 2.703, 1, 159.00017298217065], 181: [180, 293, 332493.4894205729, 5.186, 1, 159.00017298217065], 363: [361, 457, 259191.42226155597, 4.594, 1, 159.00017298217065]}, {270: [243, 481, 286541.42348734534, 5.099, 1, 164.02928496812305], 320: [243, 481, 286541.42348734534, 5.099, 1, 164.02928496812305], 359: [243, 481, 286541.42348734534, 5.099, 1, 164.02928496812305], 422: [243, 481, 286541.42348734534, 5.099, 1, 164.02928496812305], 461: [243, 481, 286541.42348734534, 5.099, 1, 164.02928496812305]}, {124: [117, 133, 131306.71557617188, 3.517, 1, 166.139010564907], 315: [314, 358, 483001.4478352865, 4.874, 1, 166.139010564907]}, {254: [118, 389, 1523533.666422526, 7.224, 1, 166.1423333451183], 323: [117, 380, 1479767.3085123696, 5.935, 1, 166.1423333451183], 340: [117, 374, 1442668.3465983071, 5.2, 1, 166.1423333451183], 272: [136, 407, 1425251.156575521, 3.396, 1, 166.1423333451183], 235: [99, 370, 1416022.4460449219, 4.868, 1, 166.1423333451183]}, {386: [384, 464, 196593.85485839844, 5.057, 1, 170.12689737981296]}, {299: [295, 368, 105384.96525065103, 4.655, 1, 170.13029991776054], 375: [374, 499, 284464.7456868489, 7.992, 1, 170.13029991776054], 418: [374, 486, 258178.83427937824, 7.852, 1, 170.13029991776054], 441: [374, 499, 284464.7456868489, 7.992, 1, 170.13029991776054]}, {324: [321, 386, 107267.80159505208, 6.084, 1, 172.98146453969318], 430: [428, 494, 115422.80836995441, 4.45, 1, 172.98146453969318], 448: [428, 485, 100578.54781087239, 4.999, 1, 172.98146453969318], 466: [428, 494, 115422.80836995441, 4.45, 1, 172.98146453969318]}, {383: [381, 469, 469058.8928222656, 4.262, 1, 173.9807028266894], 255: [253, 338, 448484.89241536456, 6.005, 1, 173.9807028266894], 339: [338, 376, 186698.32450358075, 4.187, 1, 173.9807028266894], 1: [0, 39, 184575.62890625, 4.836, 1, 173.9807028266894], 19: [1, 39, 181620.38065592447, 4.836, 1, 173.9807028266894]}, {144: [143, 328, 933237.6106770834, 2.433, 1, 173.9841824407459], 329: [328, 396, 347410.8791097005, 3.389, 1, 173.9841824407459], 1: [0, 59, 281340.0776367187, 4.95, 1, 173.9841824407459], 19: [1, 59, 278450.06465657544, 4.95, 1, 173.9841824407459], 61: [59, 100, 188454.20564778647, 2.326, 1, 173.9841824407459]}, {285: [283, 294, 196339.9563802083, 2.138, 1, 179.03191813947132], 354: [351, 361, 104687.38907877603, 2.184, 1, 179.03191813947132]}, {300: [164, 435, 6444159.957031249, 33.527, 1, 179.03549877783408], 281: [128, 382, 6008466.626698811, 12.051, 1, 179.03549877783408], 316: [180, 451, 5820834.064290364, 2.738, 1, 179.03549877783408], 265: [128, 339, 5482725.517639159, 10.669, 1, 179.03549877783408], 248: [128, 330, 5353795.846995035, 10.029, 1, 179.03549877783408]}, {176: [165, 177, 429226.7249348958, 2.129, 1, 179.03907948780966], 221: [220, 225, 147152.02734375, 2.141, 1, 179.03907948780966], 238: [237, 242, 120487.35872395833, 1.947, 1, 179.03907948780966]}, {260: [162, 312, 301543.18839518225, 8.52, 1, 180.0372992670589], 242: [162, 292, 279857.63275146484, 8.27, 1, 180.0372992670589], 208: [162, 279, 263528.46990966797, 7.307, 1, 180.0372992670589], 226: [162, 264, 241816.8056894938, 6.214, 1, 180.0372992670589], 192: [162, 234, 188577.78411356607, 4.75, 1, 180.0372992670589]}, {105: [1, 160, 3128650.480346679, 260.758, 1, 180.97954165995415], 148: [1, 161, 3137153.3819376626, 240.817, 1, 180.97954165995415], 171: [35, 306, 3451165.7857666016, 452.559, 1, 180.97954165995415], 190: [54, 325, 3467682.053955078, 363.971, 1, 180.97954165995415]}, {147: [135, 163, 107453.97322591147, 0, 1, 181.02660198776132]}, {445: [315, 499, 917227.9850565592, 5.436, 1, 181.03022251980107], 429: [315, 495, 908391.6087646484, 3.196, 1, 181.03022251980107], 255: [237, 315, 889622.0572916665, 3.111, 1, 181.03022251980107], 239: [237, 313, 872406.281738281, 3.133, 1, 181.03022251980107], 412: [315, 474, 859275.7315266927, 2.904, 1, 181.03022251980107]}, {395: [162, 430, 3207291.03342692, 17.623, 1, 181.03384312425146], 351: [162, 427, 3196581.9288736973, 17.518, 1, 181.03384312425146], 334: [162, 422, 3176855.4355672197, 16.642, 1, 181.03384312425146], 377: [162, 422, 3176855.4355672197, 16.642, 1, 181.03384312425146], 314: [162, 416, 3152247.9932250977, 15.847, 1, 181.03384312425146]}, {106: [100, 136, 121525.34355672201, 11.971, 1, 181.98130102455795], 132: [100, 137, 123248.48160807292, 14.017, 1, 181.98130102455795]}, {400: [398, 474, 279178.35350545245, 6.76, 1, 182.13422800849438], 421: [398, 473, 279734.1333414713, 6.866, 1, 182.13422800849438], 439: [398, 465, 262015.62345377603, 7.837, 1, 182.13422800849438], 460: [398, 473, 279734.1333414713, 6.866, 1, 182.13422800849438]}, {357: [356, 479, 363094.24438476557, 13.371, 1, 182.13787069305454], 478: [356, 479, 363094.24438476557, 13.371, 1, 182.13787069305454], 409: [356, 473, 355738.58768717444, 12.832, 1, 182.13787069305454], 460: [356, 473, 355738.58768717444, 12.832, 1, 182.13787069305454], 389: [356, 474, 354244.9074707031, 13.2, 1, 182.13787069305454]}, {26: [24, 88, 135602.40904744464, 3.814, 1, 186.03293352055258], 210: [205, 278, 154804.0305989583, 4.977, 1, 186.03293352055258], 279: [278, 321, 100313.41455078126, 6.177, 1, 186.03293352055258], 350: [349, 413, 165371.5911458333, 5.026, 1, 186.03293352055258], 415: [413, 455, 110003.17899576822, 3.753, 1, 186.03293352055258]}, {9: [8, 105, 204851.09912109375, 5.439, 1, 186.03665417922298], 190: [189, 350, 354296.71077473956, 3.386, 1, 186.03665417922298], 353: [350, 479, 323516.4837239583, 7.073, 1, 186.03665417922298], 406: [350, 472, 312652.24604288733, 7.057, 1, 186.03665417922298], 422: [350, 479, 323516.4837239583, 7.073, 1, 186.03665417922298]}, {370: [234, 499, 1114893.9477539062, 12.931, 1, 186.9915948613094], 352: [216, 487, 1105425.297281901, 6.066, 1, 186.9915948613094], 335: [199, 470, 1066377.7434082031, 5.175, 1, 186.9915948613094], 386: [250, 499, 1062248.6451822917, 15.316, 1, 186.9915948613094], 232: [96, 367, 1049201.48055013, 315.406, 1, 186.9915948613094]}, {108: [11, 121, 3667483.3295898438, 7.8, 1, 186.99533469320662], 406: [401, 499, 511978.0232747395, 26.792, 1, 186.99533469320662]}, {108: [102, 121, 3319596.291666666, 5.187, 1, 186.99907459990047], 130: [129, 134, 276863.8307291666, 1.271, 1, 186.99907459990047]}, {110: [101, 122, 151586.3439127604, 5.541, 1, 187.9965506357398], 126: [101, 137, 218499.85174560547, 31.668, 1, 187.9965506357398]}, {110: [100, 122, 151079.0491689046, 5.36, 1, 188.0003105667525], 126: [100, 137, 218086.8935953776, 29.759, 1, 188.0003105667525]}, {361: [225, 496, 425942.1067708334, 5.978, 1, 194.11743181841584], 378: [242, 499, 416148.67185465497, 10.792, 1, 194.11743181841584], 345: [209, 480, 409956.20373535156, 5.449, 1, 194.11743181841584], 328: [192, 463, 394971.1361897786, 4.361, 1, 194.11743181841584], 395: [259, 499, 394473.8701985677, 11.842, 1, 194.11743181841584]}, {366: [230, 499, 744781.146484375, 9.717, 1, 195.121656395641], 348: [212, 483, 712189.6137288412, 3.85, 1, 195.121656395641], 382: [246, 499, 708218.243815104, 10.503, 1, 195.121656395641], 329: [193, 464, 677400.5313313802, 4.339, 1, 195.121656395641], 400: [264, 499, 665410.7780761718, 10.716, 1, 195.121656395641]}, {66: [59, 155, 149692.43956502277, 13.329, 1, 196.95261160749567], 127: [59, 149, 141868.0799560547, 12.348, 1, 196.95261160749567], 147: [59, 155, 149692.43956502277, 13.329, 1, 196.95261160749567]}, {273: [137, 408, 568593.848470052, 4.435, 1, 196.99988543417683], 306: [170, 441, 560323.3494873046, 12.141, 1, 196.99988543417683], 323: [187, 458, 559219.6043701172, 11.149, 1, 196.99988543417683], 290: [154, 425, 555856.6126302083, 6.019, 1, 196.99988543417683], 340: [204, 475, 554896.9737548828, 8.706, 1, 196.99988543417683]}, {94: [93, 142, 137117.0804036458, 7.96, 1, 199.03944958269645], 119: [93, 130, 106754.9600423177, 6.556, 1, 199.03944958269645], 136: [93, 142, 137117.0804036458, 7.96, 1, 199.03944958269645]}, {214: [78, 349, 1635660.6927083335, 96.093, 1, 200.02512722618025], 196: [60, 331, 1634662.8492024739, 90.796, 1, 200.02512722618025], 180: [44, 315, 1632242.4589029946, 92.275, 1, 200.02512722618025], 232: [96, 367, 1631971.1267089844, 102.664, 1, 200.02512722618025], 163: [27, 298, 1626920.2783203125, 105.769, 1, 200.02512722618025]}, {463: [351, 499, 272285.33375040686, 15.715, 1, 204.0454566606564], 371: [351, 492, 263236.976196289, 10.03, 1, 204.0454566606564], 443: [351, 492, 263236.976196289, 10.03, 1, 204.0454566606564], 353: [351, 486, 256653.47892252597, 10.226, 1, 204.0454566606564], 389: [351, 469, 235952.53226725257, 12.897, 1, 204.0454566606564]}, {119: [115, 145, 118254.60757954913, 18.576, 1, 214.98801306700386]}, {6: [1, 33, 100624.18237304688, 4.396, 1, 214.9966126735217], 36: [33, 90, 188803.13142903644, 3.175, 1, 214.9966126735217], 242: [241, 318, 281466.9518229166, 2.88, 1, 214.9966126735217], 319: [318, 441, 460112.6243489584, 3.53, 1, 214.9966126735217], 450: [449, 499, 144971.89900716144, 2.418, 1, 214.9966126735217]}, {200: [197, 245, 159850.77801513672, 2.257, 1, 215.00091260577514]}, {271: [135, 406, 923790.0588378906, 3.859, 1, 217.04879239192036], 288: [152, 423, 922004.7213541666, 10.673, 1, 217.04879239192036], 255: [119, 390, 920889.3903808594, 4.43, 1, 217.04879239192036], 305: [169, 440, 914332.7718098959, 9.249, 1, 217.04879239192036], 323: [187, 458, 904710.2534993489, 10.919, 1, 217.04879239192036]}, {197: [195, 420, 375702.1154785156, 3.778, 1, 217.10088983256432]}, {115: [113, 151, 145085.2859395345, 9.429, 1, 217.10523185036095], 132: [113, 145, 134742.28148396808, 10.346, 1, 217.10523185036095], 150: [113, 151, 145085.2859395345, 9.429, 1, 217.10523185036095], 416: [415, 480, 142086.76733398438, 2.792, 1, 217.10523185036095]}, {96: [95, 151, 127509.91554768878, 15.268, 1, 218.0364242610605], 120: [95, 137, 103921.18968709308, 9.713, 1, 218.0364242610605], 136: [95, 151, 127509.91554768878, 15.268, 1, 218.0364242610605]}, {107: [98, 132, 145080.88308461505, 7.179, 1, 222.00545467994445], 123: [98, 136, 156062.66360219318, 11.788, 1, 222.00545467994445], 143: [98, 148, 167925.3042856852, 48.136, 1, 222.00545467994445]}, {355: [219, 490, 1112995.1503092446, 19.585, 1, 233.00070255228943], 335: [199, 470, 1103281.5889485679, 5.432, 1, 233.00070255228943], 373: [237, 499, 1088018.761352539, 20.925, 1, 233.00070255228943], 319: [183, 454, 1071048.7645670571, 3.113, 1, 233.00070255228943], 393: [257, 499, 1011595.6274820963, 22.125, 1, 233.00070255228943]}, {74: [72, 163, 330209.4827067057, 40.771, 1, 240.01494019352677], 103: [72, 143, 304102.1748860677, 27.535, 1, 240.01494019352677], 120: [72, 129, 241176.3037109375, 6.958, 1, 240.01494019352677], 137: [72, 144, 306201.54986572266, 34.617, 1, 240.01494019352677], 154: [72, 163, 330209.4827067057, 40.771, 1, 240.01494019352677]}, {120: [116, 129, 123287.0546875, 7.134, 1, 240.01974049233058], 137: [116, 144, 187657.5730794271, 5.18, 1, 240.01974049233058]}, {106: [101, 120, 141946.8116861979, 20.217, 1, 244.95168049231606], 124: [101, 126, 155972.9288330078, 50.244, 1, 244.95168049231606]}, {106: [101, 120, 141946.8116861979, 20.149, 1, 244.95657952592592], 124: [101, 133, 163724.80778503418, 166.226, 1, 244.95657952592592]}, {104: [95, 119, 156961.89490127563, 5.381, 1, 247.00309952033015], 123: [95, 144, 245365.8421529134, 92.065, 1, 247.00309952033015], 141: [95, 148, 246993.1936848958, 175.714, 1, 247.00309952033015]}, {113: [110, 130, 117334.93841552733, 1.94, 1, 247.00803958232055]}, {24: [22, 187, 540947.742980957, 68.178, 1, 256.9651544280733], 170: [22, 187, 540947.742980957, 68.178, 1, 256.9651544280733], 154: [22, 175, 529218.3109944662, 59.388, 1, 256.9651544280733], 137: [22, 149, 497061.6809488932, 34.591, 1, 256.9651544280733], 120: [22, 138, 452243.28103129065, 9.503, 1, 256.9651544280733]}, {108: [99, 150, 3773662.5711975098, 397.289, 1, 262.9805982180768]}, {108: [99, 149, 3770545.8243789673, 382.648, 1, 262.9858578300412]}, {105: [101, 132, 221471.58056640622, 10.189, 1, 263.98181558737645]}, {105: [99, 143, 246670.00380706787, 142.552, 1, 263.98709522368813]}, {119: [115, 133, 129248.7173055013, 5.646, 1, 266.99771673905684]}, {119: [116, 147, 162305.8036804199, 8.862, 1, 267.0030566933916]}, {109: [101, 138, 5965622.813069662, 565.075, 1, 268.99694757309214]}, {109: [101, 138, 5965622.813069662, 560.303, 1, 269.00232751204356]}, {109: [101, 129, 358314.2976710002, 18.168, 1, 269.99946972798267], 129: [99, 139, 386382.8850326538, 0, 1, 269.99946972798267]}, {109: [101, 129, 358195.7533976237, 18.218, 1, 270.00486971737723], 129: [101, 140, 386908.8703104655, 697.641, 1, 270.00486971737723]}, {111: [110, 149, 128689.13891601561, 7.491, 1, 278.9519846076232], 129: [110, 140, 114325.23404947916, 7.156, 1, 278.9519846076232]}, {102: [100, 144, 141904.426961263, 20.256, 1, 278.95756364731534], 129: [100, 140, 132548.95472208658, 8.909, 1, 278.95756364731534]}, {273: [137, 408, 315350.8868408203, 7.414, 1, 294.2056864319093], 289: [153, 424, 315223.19331868493, 9.898, 1, 294.2056864319093], 324: [188, 459, 315151.4476114909, 12.671, 1, 294.2056864319093], 341: [205, 476, 314922.1157836914, 10.699, 1, 294.2056864319093], 357: [221, 492, 314896.032063802, 10.245, 1, 294.2056864319093]}, {104: [101, 146, 236191.09684244788, 54.699, 1, 295.95253218895255]}, {104: [97, 154, 243299.1443634033, 70.233, 1, 295.95845123959623]}, {108: [99, 147, 108783.58565266927, 19.347, 1, 297.98972577284417], 143: [99, 151, 111510.58253479002, 21.245, 1, 297.98972577284417], 159: [99, 165, 117362.94368362425, 26.621, 1, 297.98972577284417]}, {105: [100, 146, 158301.0996831258, 0, 1, 322.0150337917259], 124: [100, 133, 141734.28971354166, 9.667, 1, 322.0150337917259], 140: [100, 146, 158301.0996831258, 0, 1, 322.0150337917259]}, {105: [100, 144, 157671.20259857178, 91.549, 1, 322.02147409240166], 124: [100, 133, 141734.28971354166, 9.677, 1, 322.02147409240166], 140: [100, 144, 157671.20259857178, 91.549, 1, 322.02147409240166]}, {105: [100, 122, 173338.07376098633, 7.058, 1, 329.00666813629874], 123: [100, 133, 218560.70043945312, 25.713, 1, 329.00666813629874]}, {136: [133, 280, 114455.41754150389, 6.141, 1, 333.2983013540244], 195: [133, 280, 114455.41754150389, 6.141, 1, 333.2983013540244], 265: [133, 280, 114455.41754150389, 6.141, 1, 333.2983013540244], 178: [133, 275, 109955.34939575195, 5.9, 1, 333.2983013540244], 247: [133, 275, 109955.34939575195, 5.9, 1, 333.2983013540244]}, {124: [115, 150, 155035.22016906735, 13.154, 1, 337.9634487541688]}, {104: [101, 146, 172635.71502685547, 70.632, 1, 337.97020802314387]}, {105: [100, 141, 691509.2732747395, 32.415, 1, 338.96529254140506], 140: [100, 149, 708581.6343180337, 113.279, 1, 338.96529254140506], 156: [100, 159, 714959.432647705, 165.21, 1, 338.96529254140506]}, {105: [98, 141, 692417.706471761, 31.206, 1, 338.9720718472559], 122: [97, 141, 692655.7781702677, 31.395, 1, 338.9720718472559], 140: [97, 146, 706218.7353922526, 98.54, 1, 338.9720718472559]}, {111: [100, 141, 4141735.4709879556, 397.789, 1, 344.98382927203005]}, {111: [101, 139, 4128670.5064697266, 266.31, 1, 344.99072894861547]}, {107: [100, 141, 355601.83231608075, 250.658, 1, 345.98572440378314]}, {107: [101, 139, 353026.33639017737, 309.15, 1, 345.99264411827113]}, {107: [101, 139, 111278.11294809976, 0, 1, 346.98358954753286]}, {107: [102, 135, 107255.58052571614, 33.903, 1, 346.9905292193238]}, {297: [269, 484, 138368.16614786783, 6.167, 1, 347.31685040051485], 331: [269, 484, 138368.16614786783, 6.167, 1, 347.31685040051485], 366: [269, 484, 138368.16614786783, 6.167, 1, 347.31685040051485], 405: [269, 484, 138368.16614786783, 6.167, 1, 347.31685040051485], 439: [269, 484, 138368.16614786783, 6.167, 1, 347.31685040051485]}, {292: [156, 427, 868842.9414876301, 11.135, 1, 350.32363219488576], 275: [139, 410, 864930.181640625, 9.693, 1, 350.32363219488576], 310: [174, 445, 863193.2922363281, 11.203, 1, 350.32363219488576], 242: [106, 377, 859512.3439941406, 9.826, 1, 350.32363219488576], 259: [123, 394, 859227.578531901, 10.317, 1, 350.32363219488576]}, {275: [139, 410, 860711.68359375, 9.693, 1, 350.33063866752957], 242: [106, 377, 857403.0949707031, 9.826, 1, 350.33063866752957], 259: [123, 394, 855009.080485026, 10.317, 1, 350.33063866752957], 172: [36, 307, 846341.7331542969, 9.678, 1, 350.33063866752957], 138: [2, 273, 844748.8040364583, 4.504, 1, 350.33063866752957]}, {108: [101, 138, 1713767.2412312825, 651.383, 1, 350.9968929596222]}, {108: [101, 138, 1713767.2412312825, 724.86, 1, 351.0039128974813]}, {109: [101, 118, 100854.77099609375, 8.011, 1, 352.00921092636037], 125: [101, 135, 146239.31383768716, 99.243, 1, 352.00921092636037]}, {262: [126, 397, 395881.6761881511, 8.161, 1, 355.2839219851071], 279: [143, 414, 395565.68310546875, 11.137, 1, 355.2839219851071], 313: [177, 448, 395380.545328776, 11.272, 1, 355.2839219851071], 244: [108, 379, 394132.8658854166, 8.29, 1, 355.2839219851071], 178: [42, 313, 394083.5286051432, 7.825, 1, 355.2839219851071]}, {104: [99, 144, 123931.70823160806, 60.423, 1, 360.9563697690915]}, {292: [156, 427, 839859.8080240885, 10.299, 1, 364.3434614876078], 272: [136, 407, 838732.5026041667, 9.47, 1, 364.3434614876078], 311: [175, 446, 834556.5193684895, 13.057, 1, 364.3434614876078], 327: [191, 462, 832768.9821777343, 10.602, 1, 364.3434614876078], 345: [209, 480, 829599.9341634114, 12.628, 1, 364.3434614876078]}, {56: [1, 186, 131808.9101155599, 4.193, 1, 365.3431214842938], 139: [1, 184, 130574.12009684245, 4.196, 1, 365.3431214842938], 157: [1, 186, 131808.9101155599, 4.193, 1, 365.3431214842938]}, {326: [190, 461, 403257.58447265625, 9.031, 1, 369.2953990705706], 360: [224, 495, 400715.925374349, 12.529, 1, 369.2953990705706], 309: [173, 444, 400414.4434814453, 10.609, 1, 369.2953990705706], 342: [206, 477, 400004.375406901, 12.534, 1, 369.2953990705706], 277: [141, 412, 399330.9100341797, 9.163, 1, 369.2953990705706]}, {272: [136, 407, 802233.7147623698, 9.85, 1, 394.35684866919524], 309: [173, 444, 800525.4167480469, 10.778, 1, 394.35684866919524], 292: [156, 427, 800147.9842122397, 12.305, 1, 394.35684866919524], 325: [189, 460, 798265.2244466146, 12.85, 1, 394.35684866919524], 255: [119, 390, 792063.04296875, 10.594, 1, 394.35684866919524]}, {325: [189, 460, 193888.0332438151, 6.663, 1, 399.3092110816664], 275: [139, 410, 193113.33884684244, 11.983, 1, 399.3092110816664], 341: [205, 476, 192895.99131266272, 8.354, 1, 399.3092110816664], 292: [156, 427, 192756.19274902344, 9.098, 1, 399.3092110816664], 357: [221, 492, 192572.66668701172, 8.561, 1, 399.3092110816664]}, {272: [136, 407, 949786.6223144531, 11.281, 1, 408.370911189493], 306: [170, 441, 947062.525390625, 13.524, 1, 408.370911189493], 322: [186, 457, 946287.141845703, 12.579, 1, 408.370911189493], 289: [153, 424, 943132.2777506509, 15.332, 1, 408.370911189493], 339: [203, 474, 938458.8899739583, 13.47, 1, 408.370911189493]}, {189: [187, 379, 171043.82790629068, 6.052, 1, 409.3685428521826], 274: [187, 312, 109477.4935506185, 9.588, 1, 409.3685428521826], 291: [187, 379, 171043.82790629068, 6.052, 1, 409.3685428521826]}, {105: [100, 117, 101302.61950683594, 6.497, 1, 411.0092799027422], 127: [100, 138, 151792.93344370523, 0, 1, 411.0092799027422]}, {106: [100, 144, 105565.77656046549, 0, 1, 419.97516445662563]}, {105: [100, 143, 796218.3546142576, 121.153, 1, 420.96746637746344], 126: [98, 143, 796632.8306274413, 121.235, 1, 420.96746637746344], 143: [98, 146, 800275.7250226338, 324.268, 1, 420.96746637746344]}, {105: [100, 143, 796144.531697591, 118.951, 1, 420.9758857267909], 143: [100, 148, 800944.4052352904, 476.869, 1, 420.9758857267909]}, {109: [101, 124, 1621020.7845458982, 6.414, 1, 426.9877873155397], 126: [100, 141, 2023786.4584350581, 1023.744, 1, 426.9877873155397]}, {108: [104, 109, 340149.7265625, 2.959, 1, 426.996327071286]}, {108: [101, 122, 161349.14518229166, 5.507, 1, 427.9880986425888], 124: [101, 140, 217509.68043518063, 174.021, 1, 427.9880986425888], 140: [101, 142, 218083.38409932452, 0, 1, 427.9880986425888]}, {108: [102, 122, 160090.7334798177, 5.475, 1, 427.99665840456163], 124: [102, 136, 214859.34419250485, 132.071, 1, 427.99665840456163]}, {109: [101, 121, 1501699.5751571655, 15.958, 1, 433.0075964819585]}, {108: [101, 118, 150882.20760091144, 12.207, 1, 434.0046501555535], 132: [101, 136, 197331.4396870931, 0, 1, 434.0046501555535]}, {108: [104, 118, 146957.91040039062, 11.922, 1, 434.01333024855666]}, {321: [185, 456, 397786.7659098307, 6.999, 1, 438.3751928973538], 303: [167, 438, 397520.874226888, 10.597, 1, 438.3751928973538], 287: [151, 422, 397113.03389485675, 15.133, 1, 438.3751928973538], 271: [135, 406, 396108.0392659505, 8.694, 1, 438.3751928973538], 340: [204, 475, 394621.34375, 10.474, 1, 438.3751928973538]}, {138: [136, 209, 101804.90966796874, 5.242, 1, 438.38396040121177]}, {299: [163, 434, 460649.9789632161, 14.578, 1, 452.39461169950187], 331: [195, 466, 460287.3210449219, 15.845, 1, 452.39461169950187], 281: [145, 416, 459109.2541503906, 11.45, 1, 452.39461169950187], 265: [129, 400, 458174.83146158856, 12.07, 1, 452.39461169950187], 347: [211, 482, 457854.11039225257, 11.014, 1, 452.39461169950187]}, {110: [100, 139, 114170.19509379068, 160.275, 1, 486.98537746956816]}, {106: [101, 119, 153924.6337890625, 9.503, 1, 493.0122264830123], 124: [101, 136, 194765.6066080729, 0, 1, 493.0122264830123]}, {268: [266, 444, 142332.17574564618, 5.452, 1, 496.41581147241743], 309: [266, 444, 142332.17574564618, 5.452, 1, 496.41581147241743], 372: [266, 444, 142332.17574564618, 5.452, 1, 496.41581147241743], 410: [266, 444, 142332.17574564618, 5.452, 1, 496.41581147241743], 292: [266, 434, 134422.87320454916, 5.781, 1, 496.41581147241743]}, {105: [100, 142, 815453.493367513, 474.476, 1, 502.9716335145023], 124: [100, 144, 816195.9096806843, 0, 1, 502.9716335145023]}, {105: [101, 135, 797330.9034016926, 47.702, 1, 502.98169294717246]}, {104: [101, 140, 116693.98389689127, 0, 1, 503.9785733163209]}, {109: [101, 137, 2315544.208305359, 570.196, 1, 508.9926899762755]}, {108: [101, 119, 213164.91279093424, 6.551, 1, 509.991283965147], 126: [101, 138, 299019.0690307617, 0, 1, 509.991283965147]}, {108: [103, 119, 209870.59228515622, 6.551, 1, 510.00148379082617], 126: [103, 135, 294791.90240478516, 443.263, 1, 510.00148379082617]}, {109: [102, 134, 1520937.7724405923, 722.532, 1, 515.0034179867423]}, {109: [102, 134, 1520937.7724405923, 736.733, 1, 515.013718055102]}, {109: [101, 123, 177475.069056193, 22.913, 1, 516.0034843677402], 125: [101, 136, 198350.80715433756, 0, 1, 516.0034843677402]}, {109: [103, 123, 176502.09899902344, 22.664, 1, 516.0138044374273], 125: [103, 131, 194124.93841552734, 167.63, 1, 516.0138044374273]}, {115: [102, 128, 103586.69251505533, 26.286, 1, 575.0128769966461]}, {105: [101, 136, 121058.13063557942, 24.184, 1, 578.9594073573802], 136: [101, 141, 124955.39286295572, 0, 1, 578.9594073573802]}, {106: [102, 108, 131862.763671875, 2.439, 1, 584.9651446872377]}, {106: [101, 140, 876055.4240519205, 1006.887, 1, 584.9768439901314]}, {107: [101, 139, 144823.86702601114, 0, 1, 585.9721404342581]}, {107: [101, 138, 145054.82462565103, 0, 1, 585.9838598770667]}, {110: [101, 136, 1582501.6142578123, 677.774, 1, 590.9859012253627]}, {110: [101, 136, 1582501.6142578123, 734.373, 1, 590.9977209433872]}, {109: [101, 118, 171453.1585286458, 7.561, 1, 591.9914216524732], 127: [101, 137, 241852.33288319904, 0, 1, 591.9914216524732]}, {109: [101, 118, 171453.1585286458, 7.343, 1, 592.0032614809062], 127: [101, 130, 232188.7686360677, 63.267, 1, 592.0032614809062]}, {109: [102, 120, 575109.442199707, 24.62, 1, 597.0089231484378]}, {109: [103, 120, 575667.2522786458, 24.98, 1, 597.0208633269007]}, {118: [103, 128, 103488.2247314453, 82.931, 1, 598.0127310638326]}, {118: [101, 132, 104184.73185221352, 69.717, 1, 628.9875971445721]}, {110: [102, 123, 107440.32808430988, 21.76, 1, 631.9886850189337], 126: [102, 132, 117967.87557220459, 0, 1, 631.9886850189337]}, {110: [102, 123, 107440.32808430988, 23.219, 1, 632.0013247926339], 126: [102, 127, 113268.37125651039, 0, 1, 632.0013247926339]}, {106: [101, 139, 140293.0040181478, 0, 1, 660.9584921489472]}, {106: [101, 136, 137921.45538330078, 0, 1, 660.9717113187902]}, {107: [101, 138, 657452.7327473958, 499.601, 1, 666.973924696208], 123: [101, 140, 658098.4698384602, 0, 1, 666.973924696208]}, {107: [101, 136, 655494.5935465494, 0, 1, 666.9872641747019]}, {109: [101, 124, 104568.5791422526, 6.18, 1, 667.9751262847332], 125: [101, 135, 128579.62088775635, 170.049, 1, 667.9751262847332]}, {109: [102, 124, 104147.12430826822, 6.21, 1, 667.9884857872589], 125: [102, 137, 129123.53678385417, 0, 1, 667.9884857872589]}, {121: [102, 132, 102786.28628540039, 118.665, 1, 669.9820404605969]}, {109: [101, 135, 1009440.6213429768, 674.783, 1, 672.9902633075947]}, {109: [102, 135, 1006320.6448160807, 0, 1, 673.0037231128608]}, {122: [110, 135, 103507.96405029295, 2.775, 1, 673.9870163450972]}, {109: [102, 128, 171565.841023763, 24.472, 1, 674.0004960854241], 127: [102, 132, 179474.99666341144, 0, 1, 674.0004960854241]}, {109: [103, 132, 385551.69181315094, 441.109, 1, 679.0065492487329]}, {109: [103, 131, 385247.72859700513, 0, 1, 679.0201293797179]}, {120: [103, 131, 101433.482228597, 178.314, 1, 713.9909381206319]}, {107: [101, 128, 122222.45874023436, 8.689, 1, 742.9656950246567], 124: [101, 131, 129260.18890889484, 14.831, 1, 742.9656950246567]}, {109: [101, 124, 399441.8792317708, 7.33, 1, 748.9781359906217]}, {109: [102, 124, 398624.2981363932, 7.298, 1, 748.9931155533413]}, {120: [101, 135, 109699.5451405843, 0, 1, 749.9824293762382]}, {109: [102, 134, 642962.7577438354, 484.722, 1, 754.9939320209904]}, {109: [102, 132, 639189.2651367188, 691.238, 1, 755.0090318996308]}, {109: [102, 122, 113629.32850646971, 14.28, 1, 755.9911720725225], 126: [102, 135, 131996.48869832355, 0, 1, 755.9911720725225]}, {109: [102, 122, 113629.32850646971, 14.044, 1, 756.006291895964], 126: [102, 132, 130637.39675394693, 0, 1, 756.006291895964]}, {109: [103, 122, 199334.9052734375, 38.531, 1, 761.0123852979899], 127: [103, 132, 215491.67474365234, 0, 1, 761.0123852979899]}, {109: [104, 114, 163763.9512939453, 21.232, 1, 761.0276055456958]}, {107: [101, 124, 256126.71142578122, 8.765, 1, 830.9825145440085], 124: [101, 136, 300674.5395762126, 0, 1, 830.9825145440085]}, {107: [102, 124, 255227.28869628906, 8.713, 1, 830.9991341942994], 124: [102, 129, 285283.75585937494, 29.34, 1, 830.9991341942994]}, {109: [107, 122, 264359.96468098956, 8.27, 1, 836.987119240147]}, {109: [102, 122, 333449.95570882154, 21.967, 1, 837.0038589825317]}, {109: [104, 119, 128986.3946533203, 32.133, 1, 843.018252303163]}, {109: [101, 119, 135415.78373463947, 7.419, 1, 912.9743738994296]}, {109: [101, 119, 135415.78373463947, 7.404, 1, 912.9926333869075]}, {109: [102, 121, 168621.9246419271, 21.245, 1, 919.0014924830037], 129: [102, 133, 190969.32088216147, 0, 1, 919.0014924830037]}, {109: [106, 116, 130512.08646647134, 13.946, 1, 919.0198725128535]}, {281: [145, 416, 13968891.720052082, 15.537, 1, 922.0207360386056], 299: [163, 434, 13945079.595052082, 19.303, 1, 922.0207360386056], 264: [128, 399, 13912113.52734375, 10.571, 1, 922.0207360386056], 315: [179, 450, 13872516.781249998, 14.628, 1, 922.0207360386056], 331: [195, 466, 13859130.372395836, 19.369, 1, 922.0207360386056]}, {277: [141, 412, 2637134.5836588535, 12.902, 1, 923.0170463811971], 296: [160, 431, 2633287.0865885415, 14.505, 1, 923.0170463811971], 313: [177, 448, 2619264.6272786455, 13.33, 1, 923.0170463811971], 242: [106, 377, 2618346.5135091143, 13.176, 1, 923.0170463811971], 329: [193, 464, 2617588.3225911455, 15.289, 1, 923.0170463811971]}, {236: [100, 371, 434436.51314290357, 10.317, 1, 924.0144333094446], 203: [67, 338, 431975.18729654944, 9.946, 1, 924.0144333094446], 171: [35, 306, 431152.44034830725, 12.066, 1, 924.0144333094446], 187: [51, 322, 430666.23278808594, 10.587, 1, 924.0144333094446], 219: [83, 354, 430643.2830810547, 11.791, 1, 924.0144333094446]}, {287: [151, 422, 358219.212524414, 12.817, 1, 959.9558659049055], 322: [186, 457, 356670.54162597656, 10.925, 1, 959.9558659049055], 339: [203, 474, 356572.33345540357, 9.478, 1, 959.9558659049055], 306: [170, 441, 356330.1040039062, 9.221, 1, 959.9558659049055], 251: [115, 386, 354644.869140625, 3.395, 1, 959.9558659049055]}, {1: [0, 85, 108400.12984212239, 4.345, 1, 959.9750650222235], 17: [1, 85, 106339.88052368164, 4.345, 1, 959.9750650222235]}]

In [20]:
super_list = []
for d in peak_dict:
    for k, v in d.items():  # d.items() in Python 3+
        super_list.append([k]+v)

In [25]:
[ i[0] for i in super_list]

[380,
 482,
 416,
 399,
 432,
 167,
 194,
 214,
 230,
 30,
 116,
 134,
 370,
 441,
 134,
 116,
 1,
 18,
 340,
 109,
 111,
 108,
 134,
 108,
 158,
 260,
 396,
 51,
 332,
 388,
 433,
 363,
 379,
 346,
 330,
 396,
 356,
 340,
 373,
 323,
 307,
 205,
 224,
 359,
 394,
 329,
 377,
 313,
 296,
 277,
 313,
 331,
 347,
 164,
 291,
 269,
 251,
 234,
 48,
 102,
 95,
 130,
 420,
 452,
 469,
 157,
 173,
 207,
 190,
 224,
 56,
 147,
 279,
 325,
 341,
 278,
 337,
 347,
 280,
 329,
 313,
 264,
 2,
 267,
 315,
 299,
 283,
 332,
 151,
 312,
 404,
 61,
 34,
 188,
 170,
 207,
 153,
 223,
 104,
 150,
 18,
 64,
 117,
 181,
 363,
 270,
 320,
 359,
 422,
 461,
 124,
 315,
 254,
 323,
 340,
 272,
 235,
 386,
 299,
 375,
 418,
 441,
 324,
 430,
 448,
 466,
 383,
 255,
 339,
 1,
 19,
 144,
 329,
 1,
 19,
 61,
 285,
 354,
 300,
 281,
 316,
 265,
 248,
 176,
 221,
 238,
 260,
 242,
 208,
 226,
 192,
 105,
 148,
 171,
 190,
 147,
 445,
 429,
 255,
 239,
 412,
 395,
 351,
 334,
 377,
 314,
 106,
 132,
 400,
 421,
 

In [5]:
def add(a,b):
    return a+b

In [8]:
from itertools import repeat
list(map(add, [1,2,3], repeat(4)))

[5, 6, 7]